In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [2]:
def lazyreminder():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker 
    server = "120.126.136.17"
    port = 1883 #port

    collection = [db.prediction]
    userInfo = []
    for col in collection:
        cursor = col.find({})
        dfprediction = pd.DataFrame(list(cursor))
    dfprediction = dfprediction.loc[(dfprediction['year'] == datetime.datetime.now().year) & 
                                    (dfprediction['month'] == datetime.datetime.now().month) &
                                    (dfprediction['hour'] == datetime.datetime.now().hour) &
                                    (dfprediction['day'] == datetime.datetime.now().day)]

    dfrealtimetotalsteps = pd.DataFrame({})
    while dfprediction.shape[0] != dfrealtimetotalsteps.shape[0]:
        collection = [db.realtimetotalsteps]# db.db2, db.dn2, db.james, db.leo
        userInfo = []
        for col in collection:
            cursor = col.find({})
            dfrealtimetotalsteps = pd.DataFrame(list(cursor))
    result = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values < \
                    (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
    diff = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values \
                - (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
    conn.close()
    lis = list(zip(dfprediction['user'].values,result,diff))
    for name,result,diff in lis:
        if result == False:
            mqttc = mqtt.Client()
            mqttc.connect(server, port)
            mqttc.publish(name, "你在偷懶歐! 還差" + str(int(diff)) +"步")

In [3]:
def realtimetotalstep():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.User_Info]# db.db2, db.dn2, db.james, db.leo
    userInfo = []
    for col in collection:
        cursor = col.find({})
        df_user = pd.DataFrame(list(cursor))
    collection = db.realtimetotalsteps
    collection.delete_many({})
    for username in df_user['username']:
        if username == 'pitest' or username == 'jack' or username == 'manager':
            continue
        collection = [db[username]]
        clean_data = []
        end = datetime.datetime.now().timestamp()
        start = (datetime.datetime.now().replace(hour=0,minute=0,second=0,microsecond=0)).timestamp()
        for col in collection:
            cursor = col.find({'timestamp': {'$gte': start, '$lt': end}})
            df = pd.DataFrame(list(cursor))
        df.replace('', np.nan, inplace=True)
        df.fillna(method='ffill', inplace=True)
        if 'hr_value' in df:
            df['hr_value'] = df['hr_value'].astype(float)
        if 'o2_value' in df:
            df['o2_value'] = df['o2_value'].astype(float)
        if 'latitude' in df:
            df['latitude'] = df['latitude'].astype(float)
        if 'longitude' in df:
            df['longitude'] = df['longitude'].astype(float)
        if 'step_value' in df:
            df['step_value'] = df['step_value'].astype(float)
        if 'hr_value' in df:    
            df = df[(df['hr_value'] != 0)]
        if 'step_value'  in df:
            totalstep = df['step_value'].sum()
            hourtotalstep = df.loc[df['hour'] == datetime.datetime.now().hour,['step_value']].values.sum()
        else:
            totalstep = 0
            hourtotalstep = 0
        now = datetime.datetime.now()
        dateStr = now.strftime('%Y-%m-%d-%H-%M')
        lis = dateStr.split('-')
        lis = list(map(int, lis))
        collection = db.realtimetotalsteps# db.db2, db.dn2, db.james, db.leo
        mydict = { 'year':lis[0], 'month':lis[1], 'day':lis[2], 'hour':lis[3], 'minute':lis[4],
                 'totalsteps':totalstep,'hourtotalstep':hourtotalstep,'user':username}
        
        collection.insert_one(mydict)
        conn.close()

In [4]:
schedule.every(1).minutes.do(realtimetotalstep)
schedule.every(5).minutes.do(lazyreminder)

Every 5 minutes do lazyreminder() (last run: [never], next run: 2019-05-11 16:08:53)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.prediction]# db.db2, db.dn2, db.james, db.leo
user = []

for col in collection:
    cursor = col.find({})
    prediction = pd.DataFrame(list(cursor))
timedf = prediction.loc[(prediction['month'] == datetime.datetime.now().month) 
                        & (prediction['day']== datetime.datetime.now().day) 
                        & (prediction['hour'] == datetime.datetime.now().hour)].reset_index()
timedf.drop(['index','_id'], inplace=True, axis=1)
timedf = timedf.set_index('user')
# remove duplicate index
timedf = timedf.loc[~timedf.index.duplicated(keep='last')]
for i in timedf.index:
    timedf[i] = 0
for index, row in timedf.iterrows():
    for i in timedf.index:
        timedf.loc[index,i] = timedf.loc[i,'totalsteps'] - timedf.loc[index,'totalsteps']
timedf.drop(['prediction', 'totalsteps', 'type'], inplace=True, axis=1)
timedf.reset_index(inplace=True)
records = timedf.to_dict('records')
if records != []:
    collection = db.rank
    collection.insert_many(records)
else:
    print('records', records)
conn.close()